### [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)



https://github.com/zalandoresearch/fashion-mnist


Performances : http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/


In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn, rnn
from tqdm import tqdm
from mxnet import  autograd

In [2]:
def transform_f_mnist(data, label):
    # height x weight x channel to channel x height x weight
    return(mx.nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32'))

In [3]:
test_set = gluon.data.vision.FashionMNIST(train=False, transform=transform_f_mnist)
train_set = gluon.data.vision.FashionMNIST(train=True, transform=transform_f_mnist)

In [4]:
test_set_iter  = gluon.data.DataLoader(test_set,  batch_size=120, shuffle=True)
train_set_iter = gluon.data.DataLoader(train_set, batch_size=120, shuffle=True)

In [5]:
for i, (data_x, data_y) in enumerate(train_set_iter):
    data_set_x = data_x
    data_set_y = data_y
    break

In [6]:
data_set_x.shape

(120, 1, 28, 28)

In [7]:
class f_mnist(gluon.Block):
    def __init__(self, **kwargs):
        super(f_mnist, self).__init__(**kwargs)
        with self.name_scope():
            self.conv0 = nn.Conv2D(channels=64, kernel_size=5, activation='relu')
            self.maxpool = nn.MaxPool2D()
            self.conv1 = nn.Conv2D(channels=128, kernel_size=3, activation='relu')
            self.avgpool = nn.GlobalAvgPool2D()
            self.dense = nn.Dense(10)
            
    def forward(self, inputs):
        x = self.conv0(inputs)
        x = self.maxpool(x)
        x = self.conv1(x)
        x = self.avgpool(x)
        return self.dense(x)


In [8]:
ctx = mx.gpu(0)

In [9]:
net = f_mnist()

In [10]:
def evaluate(net, data_iter, ctx):
    acc = mx.metric.Accuracy()
    for data_x, data_y in data_iter:
        data_x = data_x.as_in_context(ctx)
        data_y = data_y.as_in_context(ctx)
        with autograd.predict_mode():
            output = net(data_x)
        predictions = mx.nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=data_y)
        mx.ndarray.waitall()
    return acc.get()[1]

In [11]:
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
trainer = gluon.trainer.Trainer(net.collect_params(), 'adam')
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [12]:
net(data_set_x.as_in_context(ctx)).shape, print(net)

f_mnist(
  (conv0): Conv2D(1 -> 64, kernel_size=(5, 5), stride=(1, 1))
  (maxpool): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (conv1): Conv2D(64 -> 128, kernel_size=(3, 3), stride=(1, 1))
  (avgpool): GlobalAvgPool2D(size=(1, 1), stride=(1, 1), padding=(0, 0), ceil_mode=True)
  (dense): Dense(128 -> 10, linear)
)


((120, 10), None)

In [13]:
epoches = 20
for e in range(epoches):
    train_loss = 0
    for i, (data_x, data_y) in enumerate(tqdm(train_set_iter)):
        data = data_x.as_in_context(ctx)
        label = data_y.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss_ = loss(output, label)
        loss_.backward()
        trainer.step(data.shape[0])
        train_loss += mx.nd.mean(loss_).asscalar()
    train_acc = evaluate(net, test_set_iter,  ctx=ctx)
    test_acc  = evaluate(net, train_set_iter, ctx=ctx)
    
    print("{} : the train loss ; {}, the train accracy ; {}, the test accuracy ; {}".format(e, train_loss / len(train_set_iter), train_acc, test_acc))

  1%|          | 5/500 [00:00<00:10, 46.17it/s]

0 : the train loss ; 1.0060115902423858, the train accracy ; 0.7541, the test accuracy ; 0.7488166666666667


  1%|          | 5/500 [00:00<00:10, 49.45it/s]

1 : the train loss ; 0.6944653220772743, the train accracy ; 0.7618, the test accuracy ; 0.7600833333333333


  1%|          | 5/500 [00:00<00:10, 48.03it/s]

...

17 : the train loss ; 0.3883708582818508, the train accracy ; 0.87, the test accuracy ; 0.8675


  1%|          | 5/500 [00:00<00:10, 47.67it/s]

18 : the train loss ; 0.3825361737310886, the train accracy ; 0.8773, the test accuracy ; 0.8717


100%|██████████| 500/500 [00:07<00:00, 65.54it/s]


19 : the train loss ; 0.3778698104023933, the train accracy ; 0.8745, the test accuracy ; 0.8718666666666667
